# 逻辑回归
1. 二分类算法


In [39]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# 癌症数据集，希望做个二分类模型去判别是否为癌症
from sklearn.datasets import load_breast_cancer 
cancer = load_breast_cancer()

In [121]:
data=cancer["data"]
col = cancer['feature_names']       #提取特征的名字
x = pd.DataFrame(data,columns=col)  #那些特征的数值,30个特征，569个数据
target = cancer.target.astype(int)
y = pd.DataFrame(target,columns=['target'])#对应特征组合下的类别标签

# 划分数据集
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

model = LogisticRegression()#默认参数
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92        44
           1       0.94      0.96      0.95        70

    accuracy                           0.94       114
   macro avg       0.94      0.93      0.93       114
weighted avg       0.94      0.94      0.94       114



/Users/linshangjin/Desktop/python/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/linshangjin/Desktop/python/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


下面我们来手动实现一下逻辑回归，还是用上面的数据来分类癌症患者

In [123]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 癌症数据集，希望做个二分类模型去判别是否为癌症
from sklearn.datasets import load_breast_cancer 
cancer = load_breast_cancer()

data=cancer["data"]
col = cancer['feature_names']       #提取特征的名字
X = pd.DataFrame(data,columns=col)  #那些特征的数值,30个特征，569个数据
target = cancer.target.astype(int)
y = pd.DataFrame(target,columns=['target'])#对应特征组合下的类别标签
y=y.to_numpy()

X_b=np.concatenate([np.ones((len(y),1)),X],axis=1)

# 划分数据集
x_train,x_test,y_train,y_test=train_test_split(X_b,y,test_size=0.2,random_state=0)

In [149]:
# 学习率衰减策略，迭代到后面学习率逐渐变小
t0,t1=5,50
def learning_schedule(t):
    return t0/(t+t1)

# 迭代次数
n_epoch=10000
# 初始化待求参数
theta=np.ones((31,1))
# 使用样本数
m=len(y_test)


for epoch in range(n_epoch):
    gradients= -1/m * (y_train.T-theta.T.dot(x_train.T)).dot(x_train)
    eta=learning_schedule(epoch)
    theta=theta - eta*gradients.T

# 验证集检验
y_test_pred=(theta.T.dot(x_test.T)>0).astype(int)

# 计算准确率  
accuracy = np.mean(y_test_pred == y_test.T)  
print(f'Accuracy: {accuracy * 100:.2f}%') 

/var/folders/1n/4skwfl_n6gq3hxvc100399q80000gn/T/ipykernel_92359/3143669602.py:17: RuntimeWarning: invalid value encountered in subtract
  theta=theta - eta*gradients.T


Accuracy: 41.23%


In [145]:
x_train

array([[ 1.     , 10.05   , 17.53   , ...,  0.06499,  0.2894 ,  0.07664],
       [ 1.     , 10.8    , 21.98   , ...,  0.07485,  0.2965 ,  0.07662],
       [ 1.     , 16.14   , 14.86   , ...,  0.1129 ,  0.2778 ,  0.07012],
       ...,
       [ 1.     ,  9.436  , 18.32   , ...,  0.05052,  0.2454 ,  0.08136],
       [ 1.     ,  9.72   , 18.22   , ...,  0.     ,  0.1909 ,  0.06559],
       [ 1.     , 11.51   , 23.93   , ...,  0.09653,  0.2112 ,  0.08732]])